# HW12 - Misson to Mars

In this assignment, you will build a web application that scrapes various websites for data related to the Mission to Mars and displays the information in a single HTML page. The following outlines what you need to do.

## Step 1 - Scraping

<hr>

### NASA Mars News

In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
# import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
# news_url = 'https://mars.nasa.gov/news'
# response_news = requests.get(news_url)
# soup_news = bs(response_news.text, 'html.parser')

In [3]:
# Splinter setup
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======


[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [/Users/ZTC/.wdm/drivers/chromedriver/mac64/89.0.4389.23/chromedriver] found in cache


In [4]:
# Navigate browser
news_url = 'https://mars.nasa.gov/news'
browser.visit(news_url)

Collect the latest new article info.

In [5]:
news_html = browser.html
soup_news = bs(news_html, "html.parser")

In [10]:
# Scrape results
results = soup_news.find_all('li', class_='slide')

latest_news_title = results[0].find('div', class_='content_title')
latest_news_body = results[0].find('div', class_='article_teaser_body')

In [11]:
news_title = latest_news_title.text
news_p = latest_news_body.text
news_link = latest_news_title.a['href']

In [12]:
# Check news title and paragraph is scrapped correctly and reflects the latest news article
print(news_title)
print(news_p)
news_link = 'https://mars.nasa.gov' + news_link
print(news_link)

Another First: Perseverance Captures the Sounds of Driving on Mars
NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.
https://mars.nasa.gov/news/8892/another-first-perseverance-captures-the-sounds-of-driving-on-mars/


### JPL Mars Space Images - Featured Image

In [9]:
# Navigate browser
jpl_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
browser.visit(jpl_url)

Get the URL of the current featured image.

In [10]:
jpl_html = browser.html
soup_jpl = bs(jpl_html, 'html.parser')

# Find the anchor tag of the featured image
featured_img = soup_jpl.find_all('a', class_='showimg')[0]['href']

featured_image_url = jpl_url.replace('index.html', featured_img)
featured_image_url

'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars3.jpg'

### Mars Facts

Scrape the table on Mars Facts webpage containing facts about the planet including Diameter, Mass, etc.

In [11]:
# Use pandas to scrape the table containing facts about Mars
facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [12]:
# Change column headers
df = tables[0]
df = df.rename(columns={0: 'Attributes', 1: 'Facts'})
df

,Attributes,Facts
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [13]:
# Convert the dataframe to html
df.to_html('table.html', index=False, justify='left')

### Mars Hemispheres

In [21]:
# Navigate browser
hem_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hem_url)

Obtain URLs of high resolution images for each of Mar's 4 hemispheres.

In [23]:
hem_html = browser.html
soup_hem = bs(hem_html, 'html.parser')

In [26]:
# Create an empty list to append dictionaries
hemisphere_image_urls = []

# Find divs
hems = soup_hem.find_all('div', class_='description')

counter = 1

for hem in hems:
    # Scrape hemisphere names
    title = hem.a.text.replace(' Enhanced', '')
    
    # Use Splinter to click into each listed hemisphere to get high resolution image URL
    url_enhanced = 'https://astrogeology.usgs.gov' + hem.a['href']
    
    browser.visit(url_enhanced)
    enhanced_html = browser.html
    soup_enhanced = bs(enhanced_html, 'html.parser')

    # Get URL
    src_url = soup_enhanced.find('img', class_='wide-image')['src']
    image_url = 'https://astrogeology.usgs.gov' + src_url
    
    print('-------------')
    print(f'Hemisphere {counter}\'s title and URL retrived...')
    
    counter += 1
    
    # Update list to create a list of dictionaries
    hemisphere_image_urls.append(
        {'title': title,
         'image_url': image_url
        }
    )


-------------
Hemisphere 1's title and URL retrived...
-------------
Hemisphere 2's title and URL retrived...
-------------
Hemisphere 3's title and URL retrived...
-------------
Hemisphere 4's title and URL retrived...


In [27]:
# Check if dictionary has all the info needed
print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'image_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'image_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'image_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'image_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]


In [18]:
# browser.quit()